In [1]:
import numpy as np
import sympy as sp
from sympy import symbols, pprint
import matplotlib.pyplot as plt
import matplotlib.animation as animation


# Pre: Estimate of `C=O` k and `C=C` K spring Constants


## Overview

In this section, we investigate a model for a 1D linear chain of atoms. The system uses two standard approximations:
Harmonic approximation and Nearest-neighbor interaction.

### Equations of Motion

For small displacements from equilibrium, the Newtonian equations of motion for the \( i \)-th atom are:

$$
m_i \, \Delta\ddot{x}_i = - (k_{i-1} + k_i) \Delta x_i + k_{i-1} \Delta x_{i-1} + k_i \Delta x_{i+1}
$$

This leads to a matrix formulation:

$$
\mathbf{M} \, \Delta\ddot{\mathbf{x}} = - \mathbf{K} \, \Delta\mathbf{x}
$$

where $\mathbf{M}$ is a diagonal matrix of atomic masses and $\mathbf{K}$ is the symmetric stiffness matrix of spring constants, the equation becomes:

$$
\Delta\ddot{\mathbf{x}} = - \mathbf{M}^{-1} \mathbf{K} \, \Delta\mathbf{x} \tag{1}
$$

The matrix $ \mathbf{M}^{-1} \mathbf{K} $ is guaranteed to be diagonalizable with real, non-negative eigenvalues (since $ \mathbf{K} $ is positive semi-definite), making it suitable for modal analysis.

Solving the eigenvalue problem:

$$
\det\left( \mathbf{M}^{-1} \mathbf{K} - \omega^2 \mathbf{I} \right) = 0
$$

yields the*normal mode frequencies $ \omega $ and corresponding eigenvectors (modes). This forms the basis of vibrational analysis in the symbolic class described below.

---

## SymbolicMolecularChain and SymbolicEigenSolver: SymPy-Based

> ⚠️ **Note**: This class is derived for symbolically derive equations of motion (EOM) and eigenfrequencies for linear molecular chains.  
> It was implemented using `sympy`, which is **not available** in the Docker image, so all symbolic analysis was executed in the **host environment**.

## MolecularChainNumeric: numpy-Based
The `SymbolicMolecularChain` and `SymbolicEigenSolver` used to estimate spring constants from empirical data (e.g., C–C and C=O stretching frequencies).

 After deriving the symbolic form of the equations of motion using `sympy`, we now transition to a fully numerical implementation using `numpy`: `MolecularChainNumeric`, for practical computation of eigenfrequencies and modes.





### Input Parameters

- `structure`: A string representing the linear molecular configuration using atomic symbols and bond types.  
  ❗ **No spaces allowed.**  
  For example:  
  - `"O=C=O"` for a symmetric molecule.  
  - `"O=C-C-C=O"` to distinguish between double (`=`) and single (`-`) bonds in a C₃O₂-like structure.  
    _(Note: this is not the exact chemical structure; it is a schematic input format.)_

- `mass_dict`: A dictionary assigning atomic masses to each element.  
  Example:
  ```python
  {'O': m, 'C': M}

In [2]:


class SymbolicMolecularChain:
    def __init__(self, structure, mass_dict, spring_dict):
        self.structure = structure
        self.mass_dict = mass_dict
        self.spring_dict = spring_dict
        self.atoms, self.bonds = self._parse_structure(structure)
        self.N = len(self.atoms)

    def _parse_structure(self, structure):
        atoms, bonds = [], []
        i = 0
        while i < len(structure):
            if structure[i].isalpha():
                atoms.append(structure[i])
                if i + 1 < len(structure) and structure[i + 1] in self.spring_dict:
                    bonds.append(structure[i + 1])
                    i += 2
                else:
                    i += 1
            else:
                i += 1
        return atoms, bonds

    def build_matrix(self):
        m = [self.mass_dict[a] for a in self.atoms]
        k = [self.spring_dict[b] for b in self.bonds]

        A = sp.zeros(self.N)
        for i in range(self.N):
            if i > 0:
                A[i, i - 1] = -k[i - 1] / m[i]
                A[i, i] += k[i - 1] / m[i]
            if i < self.N - 1:
                A[i, i + 1] = -k[i] / m[i]
                A[i, i] += k[i] / m[i]
        return A

In [3]:
class SymbolicEigenSolver:
    def __init__(self, matrix):
        self.matrix = matrix
        self.N = matrix.shape[0]
        self.lambda_sym = sp.Symbol('lambda')

    def characteristic_polynomial(self):
        return (self.matrix - self.lambda_sym * sp.eye(self.N)).det()

    def eigenvalues(self):
        return sp.solve(self.characteristic_polynomial(), self.lambda_sym)

    def diagonalize(self):
        return self.matrix.diagonalize()


#### Example CO2

In [4]:
m, M, k, K = symbols("m M k K")

CO2 = SymbolicMolecularChain("O=C=O", {'O': m, 'C': M}, {'=': k,'-':K})

Matrix_EignValue_CO2 = CO2.build_matrix()

In [5]:

solver_CO2 = SymbolicEigenSolver(Matrix_EignValue_CO2)
eigenvals = solver_CO2.eigenvalues()
pprint(eigenvals)

⎡   k  M⋅k + 2⋅k⋅m⎤
⎢0, ─, ───────────⎥
⎣   m      M⋅m    ⎦


## Numerical Estimation for spring constants based on 

http://www2.ess.ucla.edu/~schauble/MoleculeHTML/CO2_html/CO2_page.html

https://orgchemboulder.com/Spectroscopy/irtutor/alkenesir.shtml

In [6]:

c_m_s = 2.99792458e8 

# Atomic masses in kg
u = 1.66053906660e-27
m = 15.999 * u
M = 12.011 * u


In [7]:

#transitional mode
w_1=0

#mid_static_mode
w_2=135360*c_m_s*2*np.pi

#anti_mode
w_3=239630*c_m_s*2*np.pi


k_from_w2 = w_2**2 * m
k_from_w3 = w_3**2 / (1/m + 2/M)
k_from_w2, k_from_w3


(1727.1249969385876, 1477.2810284076643)

In [8]:

k_avg=(k_from_w2+ k_from_w3)/2
k_avg




1602.203012673126

In [9]:

m, M, k, K = symbols("m M k K")

C2 = SymbolicMolecularChain("C=C", {'O': m, 'C': M}, {'=': k,'-':K})

Matrix_EignValue_C2 = C2.build_matrix()


solver_C2 = SymbolicEigenSolver(Matrix_EignValue_C2)
eigenvals = solver_C2.eigenvalues()
pprint(eigenvals)

⎡   2⋅k⎤
⎢0, ───⎥
⎣    M ⎦


In [10]:
M = 12.011 * u

w_cc_1=164000
w_cc_2=168000

w_cc=(w_cc_1+w_cc_2)/2
w_cc= w_cc*c_m_s*2*np.pi


(w_cc**2)* M/2

975.0246320574357

K is stiffer than k, so our estimates are consistent. Also they are consistent with those sources! 

# Problem 3 

In [11]:
'''based on the estimates we have so far I will use 
'''

u = 1.66053906660e-27

m = 15.999 * u
M = 12.011 * u

k=1602.20
K=975.02 


In [12]:
# now I will use numpy



class MolecularChainNumeric:
    def __init__(self, structure, mass_dict, spring_dict):
        self.structure = structure
        self.mass_dict = mass_dict
        self.spring_dict = spring_dict
        self.atoms, self.bonds = self._parse_structure(structure)
        self.N = len(self.atoms)

    def _parse_structure(self, structure):
        atoms, bonds = [], []
        i = 0
        while i < len(structure):
            if structure[i].isalpha():
                atoms.append(structure[i])
                if i + 1 < len(structure) and structure[i + 1] in self.spring_dict:
                    bonds.append(structure[i + 1])
                    i += 2
                else:
                    i += 1
            else:
                i += 1
        return atoms, bonds

    def build_matrix(self):
        m = np.array([self.mass_dict[a] for a in self.atoms])
        k = np.array([self.spring_dict[b] for b in self.bonds])

        A = np.zeros((self.N, self.N))
        for i in range(self.N):
            if i > 0:
                A[i, i - 1] = -k[i - 1] / m[i]
                A[i, i] += k[i - 1] / m[i]
            if i < self.N - 1:
                A[i, i + 1] = -k[i] / m[i]
                A[i, i] += k[i] / m[i]
        return A


class NumericEigenSolver:
    def __init__(self, matrix):
        self.matrix = matrix

    def solve(self):
        # eigenvalues are ω², eigenvectors are mode shapes
        eigenvals, eigenvecs = np.linalg.eigh(self.matrix)
        freqs = np.sqrt(np.clip(eigenvals, 0, None))  # ω = sqrt(λ)
        return freqs, eigenvecs


# (A) C3O2


In [13]:
C3O2 = MolecularChainNumeric("O=C-C-C=O", {'O': m, 'C': M}, {'=': k,'-': K})
A = C3O2.build_matrix()

solver = NumericEigenSolver(A)
square_frequencies, modes = solver.solve()
frequencies=np.sqrt(square_frequencies)

for i, (freq, mode) in enumerate(zip(np.sqrt(frequencies), modes.T), start=1):
    print(f"Frequency {i}: {freq:.3e} Hz")
    print(f"Mode {i}: {np.round(mode, 4)} m\n\n")

Frequency 1: 0.000e+00 Hz
Mode 1: [0.677  0.5444 0.3743 0.237  0.2213] m


Frequency 2: 3.448e+03 Hz
Mode 2: [ 0.3713  0.1865 -0.1933 -0.4942 -0.7387] m


Frequency 3: 4.105e+03 Hz
Mode 3: [ 0.399  -0.1013 -0.7562 -0.1629  0.4819] m


Frequency 4: 4.513e+03 Hz
Mode 4: [-0.3434  0.4775  0.1464 -0.6998  0.3779] m


Frequency 5: 4.622e+03 Hz
Mode 5: [ 0.3558 -0.6561  0.4789 -0.4281  0.1743] m





# (B) C9O2


In [14]:
C9O2 = MolecularChainNumeric("O=C-C-C-C-C-C-C-C-C=O", {'O': m, 'C': M}, {'=': k,'-': K})
A = C9O2.build_matrix()

solver = NumericEigenSolver(A)
square_frequencies, modes = solver.solve()

for i, (freq, mode) in enumerate(zip(np.sqrt(square_frequencies), modes.T), start=1):
    print(f"Frequency {i}: {freq:.3e} Hz")
    print(f"Mode {i}: {np.round(mode, 4)} m\n\n")

Frequency 1: 0.000e+00 Hz
Mode 1: [0.6475 0.5266 0.3821 0.2769 0.2002 0.1441 0.1027 0.072  0.0486 0.0302
 0.0279] m


Frequency 2: 8.459e+06 Hz
Mode 2: [ 0.227   0.156   0.0229 -0.1126 -0.2363 -0.3353 -0.3991 -0.4212 -0.3991
 -0.3353 -0.3664] m


Frequency 3: 1.130e+07 Hz
Mode 3: [ 0.2539  0.139  -0.0962 -0.2993 -0.4024 -0.3712 -0.2161  0.0112  0.2348
  0.3799  0.5209] m


Frequency 4: 1.362e+07 Hz
Mode 4: [ 0.2655  0.0855 -0.2705 -0.4359 -0.2943  0.0546  0.365   0.4184  0.177
 -0.189  -0.4406] m


Frequency 5: 1.567e+07 Hz
Mode 5: [ 2.545e-01 -1.000e-04 -4.184e-01 -3.202e-01  1.733e-01  4.529e-01
  1.734e-01 -3.201e-01 -4.185e-01 -2.000e-04  3.389e-01] m


Frequency 6: 1.739e+07 Hz
Mode 6: [ 0.2354 -0.0915 -0.4573  0.033   0.4615  0.026  -0.4582 -0.0846  0.4474
  0.1418 -0.274 ] m


Frequency 7: 1.878e+07 Hz
Mode 7: [ 0.2161 -0.1723 -0.3721  0.3748  0.1682 -0.4663  0.0855  0.4198 -0.3139
 -0.249   0.2344] m


Frequency 8: 1.984e+07 Hz
Mode 8: [ 0.1973 -0.2328 -0.2013  0.4686 -0.3478 -